In [201]:
import mysql.connector
import pymysql
import pandas as pd
from sqlalchemy import create_engine

In [202]:
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load environment variables from .env file
load_dotenv()


# Get DB credentials from environment variables
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT") or 3306)
DB_NAME = os.getenv("DB_NAME")
print(f"DB_USER='{DB_USER}'")
print(f"DB_PASSWORD='{DB_PASSWORD}'")
print(f"DB_HOST='{DB_HOST}'")
print(f"DB_PORT='{DB_PORT}'")
print(f"DB_NAME='{DB_NAME}'")

from urllib.parse import quote_plus

password_encoded = quote_plus(DB_PASSWORD)
engine = create_engine(f"mysql+pymysql://{DB_USER}:{password_encoded}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

conn = engine.connect()

print("✅ Database connection successful.")

DB_USER='root'
DB_PASSWORD='Istatieh2005@'
DB_HOST='localhost'
DB_PORT='3306'
DB_NAME='sakila'
✅ Database connection successful.


In [203]:


query = """
SELECT 
    p.payment_id,
    p.staff_id,
    p.rental_id,
    r.inventory_id,          
    DATE(p.payment_date) AS payment_date,
    p.amount,
    s.first_name,
    s.last_name,
    r.rental_date,
    r.customer_id,
    s.store_id

FROM payment p
JOIN staff s ON p.staff_id = s.staff_id
JOIN rental r ON p.rental_id = r.rental_id
"""

df = pd.read_sql(query, engine)
print(df.head(20))



    payment_id  staff_id  rental_id  inventory_id payment_date  amount  \
0            1         1         76          3021   2005-05-25    2.99   
1            2         1        573          4020   2005-05-28    0.99   
2            3         1       1185          2785   2005-06-15    5.99   
3            6         1       1725           726   2005-06-16    4.99   
4            7         1       2308           197   2005-06-18    4.99   
5            9         1       3284          4566   2005-06-21    3.99   
6           11         1       4611          3486   2005-07-08    5.99   
7           12         1       5244          3726   2005-07-09    4.99   
8           13         1       5326           797   2005-07-09    4.99   
9           14         1       6163          1330   2005-07-11    7.99   
10          16         1       7841          1092   2005-07-28    4.99   
11          18         1       8074          1558   2005-07-28    0.99   
12          24         1      11367   

In [204]:
df.describe()

,payment_id,staff_id,rental_id,inventory_id,amount,rental_date,customer_id,store_id
count,16044.000000,16044.000000,16044.000000,16044.000000,16044.000000,16044,16044.000000,16044.000000
mean,8024.483732,1.498005,8025.371478,2291.842558,4.201356,2005-07-23 08:12:53.217526912,297.143169,1.498005
min,1.000000,1.000000,1.000000,1.000000,0.000000,2005-05-24 22:53:30,1.000000,1.000000
25%,4012.750000,1.000000,4013.750000,1154.000000,2.990000,2005-07-07 00:58:40.500000,148.000000,1.000000
50%,8024.500000,1.000000,8025.500000,2291.000000,3.990000,2005-07-28 16:04:32.500000,296.000000,1.000000
75%,12036.250000,2.000000,12037.250000,3433.000000,4.990000,2005-08-17 21:16:23,446.000000,2.000000
max,16049.000000,2.000000,16049.000000,4581.000000,11.990000,2006-02-14 15:16:03,599.000000,2.000000
std,4632.695501,0.500012,4632.777249,1322.210643,2.362961,NaN,172.453136,0.500012


In [205]:
df.dtypes

payment_id               int64
staff_id                 int64
rental_id                int64
inventory_id             int64
payment_date            object
amount                 float64
first_name              object
last_name               object
rental_date     datetime64[ns]
customer_id              int64
store_id                 int64
dtype: object

In [206]:
print(df.isnull().sum())
df=df.dropna()

payment_id      0
staff_id        0
rental_id       0
inventory_id    0
payment_date    0
amount          0
first_name      0
last_name       0
rental_date     0
customer_id     0
store_id        0
dtype: int64


In [207]:
df.duplicated().sum()

0

In [208]:
df['year'] = pd.to_datetime(df['payment_date']).dt.year
df['month'] = pd.to_datetime(df['payment_date']).dt.month
df['day'] = pd.to_datetime(df['payment_date']).dt.day
df.dtypes

payment_id               int64
staff_id                 int64
rental_id                int64
inventory_id             int64
payment_date            object
amount                 float64
first_name              object
last_name               object
rental_date     datetime64[ns]
customer_id              int64
store_id                 int64
year                     int32
month                    int32
day                      int32
dtype: object

In [209]:
df

,payment_id,staff_id,rental_id,inventory_id,payment_date,amount,first_name,last_name,rental_date,customer_id,store_id,year,month,day
0,1,1,76,3021,2005-05-25,2.99,Mike,Hillyer,2005-05-25 11:30:37,1,1,2005,5,25
1,2,1,573,4020,2005-05-28,0.99,Mike,Hillyer,2005-05-28 10:35:23,1,1,2005,5,28
2,3,1,1185,2785,2005-06-15,5.99,Mike,Hillyer,2005-06-15 00:54:12,1,1,2005,6,15
3,6,1,1725,726,2005-06-16,4.99,Mike,Hillyer,2005-06-16 15:18:57,1,1,2005,6,16
4,7,1,2308,197,2005-06-18,4.99,Mike,Hillyer,2005-06-18 08:41:48,1,1,2005,6,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16039,16042,2,9679,3819,2005-07-31,2.99,Jon,Stephens,2005-07-31 06:41:19,599,2,2005,7,31
16040,16043,2,11522,2033,2005-08-17,3.99,Jon,Stephens,2005-08-17 00:05:05,599,2,2005,8,17
16041,16047,2,15590,4048,2005-08-23,8.99,Jon,Stephens,2005-08-23 06:09:44,599,2,2005,8,23
16042,16048,2,15719,3990,2005-08-23,2.99,Jon,Stephens,2005-08-23 11:08:46,599,2,2005,8,23


In [210]:
#loading staff
dim_staff=df[["staff_id","first_name","last_name","store_id"]].drop_duplicates()
dim_staff.to_sql('dim_staff',con=engine,if_exists='append',index=False)


2

In [211]:
df

,payment_id,staff_id,rental_id,inventory_id,payment_date,amount,first_name,last_name,rental_date,customer_id,store_id,year,month,day
0,1,1,76,3021,2005-05-25,2.99,Mike,Hillyer,2005-05-25 11:30:37,1,1,2005,5,25
1,2,1,573,4020,2005-05-28,0.99,Mike,Hillyer,2005-05-28 10:35:23,1,1,2005,5,28
2,3,1,1185,2785,2005-06-15,5.99,Mike,Hillyer,2005-06-15 00:54:12,1,1,2005,6,15
3,6,1,1725,726,2005-06-16,4.99,Mike,Hillyer,2005-06-16 15:18:57,1,1,2005,6,16
4,7,1,2308,197,2005-06-18,4.99,Mike,Hillyer,2005-06-18 08:41:48,1,1,2005,6,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16039,16042,2,9679,3819,2005-07-31,2.99,Jon,Stephens,2005-07-31 06:41:19,599,2,2005,7,31
16040,16043,2,11522,2033,2005-08-17,3.99,Jon,Stephens,2005-08-17 00:05:05,599,2,2005,8,17
16041,16047,2,15590,4048,2005-08-23,8.99,Jon,Stephens,2005-08-23 06:09:44,599,2,2005,8,23
16042,16048,2,15719,3990,2005-08-23,2.99,Jon,Stephens,2005-08-23 11:08:46,599,2,2005,8,23


In [212]:
# Create a unique date_id (you can use the date as the ID if it's unique)
dim_date = df[['payment_date', 'month', 'year']].drop_duplicates()
dim_date['date_id'] = pd.to_datetime(dim_date['payment_date']).dt.strftime('%Y%m%d').astype(int)
dim_date = dim_date[['date_id', 'payment_date', 'month', 'year']]
dim_date.columns = ['date_id', 'full_date', 'month', 'year']
dim_date.to_sql('dim_date', con=engine, if_exists='append', index=False)







41

In [213]:
inventory_query = """
SELECT 
    i.inventory_id,
    i.film_id,
    f.title ,
    i.store_id,
    s.address_id,
    s.manager_staff_id,
    a.address,
    f.release_year,
    f.language_id,
    i.last_update
FROM inventory i
JOIN film f ON i.film_id = f.film_id
JOIN store s ON i.store_id = s.store_id
JOIN address a ON s.address_id = a.address_id
"""
df_inventory=pd.read_sql(inventory_query,engine)
df_inventory = df_inventory.drop_duplicates().dropna()
df_inventory


,inventory_id,film_id,title,store_id,address_id,manager_staff_id,address,release_year,language_id,last_update
0,1,1,ACADEMY DINOSAUR,1,1,1,47 MySakila Drive,2006,1,2006-02-15 05:09:17
1,2,1,ACADEMY DINOSAUR,1,1,1,47 MySakila Drive,2006,1,2006-02-15 05:09:17
2,3,1,ACADEMY DINOSAUR,1,1,1,47 MySakila Drive,2006,1,2006-02-15 05:09:17
3,4,1,ACADEMY DINOSAUR,1,1,1,47 MySakila Drive,2006,1,2006-02-15 05:09:17
4,16,4,AFFAIR PREJUDICE,1,1,1,47 MySakila Drive,2006,1,2006-02-15 05:09:17
...,...,...,...,...,...,...,...,...,...,...
4576,4573,999,ZOOLANDER FICTION,2,2,2,28 MySQL Boulevard,2006,1,2006-02-15 05:09:17
4577,4578,1000,ZORRO ARK,2,2,2,28 MySQL Boulevard,2006,1,2006-02-15 05:09:17
4578,4579,1000,ZORRO ARK,2,2,2,28 MySQL Boulevard,2006,1,2006-02-15 05:09:17
4579,4580,1000,ZORRO ARK,2,2,2,28 MySQL Boulevard,2006,1,2006-02-15 05:09:17


In [214]:
df

,payment_id,staff_id,rental_id,inventory_id,payment_date,amount,first_name,last_name,rental_date,customer_id,store_id,year,month,day
0,1,1,76,3021,2005-05-25,2.99,Mike,Hillyer,2005-05-25 11:30:37,1,1,2005,5,25
1,2,1,573,4020,2005-05-28,0.99,Mike,Hillyer,2005-05-28 10:35:23,1,1,2005,5,28
2,3,1,1185,2785,2005-06-15,5.99,Mike,Hillyer,2005-06-15 00:54:12,1,1,2005,6,15
3,6,1,1725,726,2005-06-16,4.99,Mike,Hillyer,2005-06-16 15:18:57,1,1,2005,6,16
4,7,1,2308,197,2005-06-18,4.99,Mike,Hillyer,2005-06-18 08:41:48,1,1,2005,6,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16039,16042,2,9679,3819,2005-07-31,2.99,Jon,Stephens,2005-07-31 06:41:19,599,2,2005,7,31
16040,16043,2,11522,2033,2005-08-17,3.99,Jon,Stephens,2005-08-17 00:05:05,599,2,2005,8,17
16041,16047,2,15590,4048,2005-08-23,8.99,Jon,Stephens,2005-08-23 06:09:44,599,2,2005,8,23
16042,16048,2,15719,3990,2005-08-23,2.99,Jon,Stephens,2005-08-23 11:08:46,599,2,2005,8,23


In [215]:
dim_film=df_inventory[['film_id','title','release_year',"language_id"]].drop_duplicates()
dim_film.to_sql('dim_film',con=engine,if_exists='append',index=False)


958

In [216]:
dim_store=df_inventory[['store_id','manager_staff_id','address_id']].drop_duplicates()
dim_store.to_sql('dim_store',con=engine,if_exists='append',index=False)

2

In [217]:
dim_rental=df[["rental_id","rental_date","inventory_id","customer_id"]].drop_duplicates()
dim_rental.to_sql('dim_rental',con=engine,if_exists='append',index=False)

16044

In [218]:
# Load dim_date
dim_date = pd.read_sql("SELECT date_id, full_date FROM dim_date", engine)

# Ensure types match
df_inventory['last_update'] = pd.to_datetime(df_inventory['last_update']).dt.date
dim_date['full_date'] = pd.to_datetime(dim_date['full_date']).dt.date

# Join to get date_id
df_inventory = df_inventory.merge(dim_date, left_on='last_update', right_on='full_date', how='left')




In [219]:
fact_inventory = df_inventory[['date_id', 'film_id', 'store_id']]
fact_inventory.to_sql('fact_inventory', con=engine, if_exists='append', index=False)

4581

In [220]:
df

,payment_id,staff_id,rental_id,inventory_id,payment_date,amount,first_name,last_name,rental_date,customer_id,store_id,year,month,day
0,1,1,76,3021,2005-05-25,2.99,Mike,Hillyer,2005-05-25 11:30:37,1,1,2005,5,25
1,2,1,573,4020,2005-05-28,0.99,Mike,Hillyer,2005-05-28 10:35:23,1,1,2005,5,28
2,3,1,1185,2785,2005-06-15,5.99,Mike,Hillyer,2005-06-15 00:54:12,1,1,2005,6,15
3,6,1,1725,726,2005-06-16,4.99,Mike,Hillyer,2005-06-16 15:18:57,1,1,2005,6,16
4,7,1,2308,197,2005-06-18,4.99,Mike,Hillyer,2005-06-18 08:41:48,1,1,2005,6,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16039,16042,2,9679,3819,2005-07-31,2.99,Jon,Stephens,2005-07-31 06:41:19,599,2,2005,7,31
16040,16043,2,11522,2033,2005-08-17,3.99,Jon,Stephens,2005-08-17 00:05:05,599,2,2005,8,17
16041,16047,2,15590,4048,2005-08-23,8.99,Jon,Stephens,2005-08-23 06:09:44,599,2,2005,8,23
16042,16048,2,15719,3990,2005-08-23,2.99,Jon,Stephens,2005-08-23 11:08:46,599,2,2005,8,23


In [221]:
# Step 1: Extract year and month from payment_date
df['payment_date'] = pd.to_datetime(df['payment_date'])
df['year'] = df['payment_date'].dt.year
df['month'] = df['payment_date'].dt.month

# Step 2: Group by staff_id, rental_id, year, and month to get monthly totals
monthly_agg = df.groupby(['staff_id', 'rental_id', 'year', 'month'], as_index=False)['amount'].sum()
monthly_agg.rename(columns={'amount': 'monthly_payment_total'}, inplace=True)

# Step 3: Join with dim_date to get date_id
dim_date = pd.read_sql("SELECT date_id, year, month FROM dim_date", engine)
fact = pd.merge(monthly_agg, dim_date, on=['year', 'month'], how='left')

# Step 4: Select and reorder columns for fact table
fact = fact[['staff_id', 'rental_id', 'date_id', 'monthly_payment_total']]

# Step 5: Load into fact_monthly_payment table
fact.to_sql('fact_monthly_payment', con=engine, if_exists='append', index=False)


178704